### Traditional use of **getter** and **setter**

In [1]:
class OldResistor(object):
    
    def __init__(self, ohms):
        self._ohms = ohms
        
    def get_ohms(self):
        return self._ohms
    
    def set_ohms(self, ohms):
        self._ohms = ohms

In [2]:
r0 = OldResistor(50e3)
print("Before: %5r" % r0.get_ohms())

Before: 50000.0


In [4]:
r0.set_ohms(10e3)
print("After: %5r" % r0.get_ohms())

After: 10000.0


This is correct, but not **pythonic** !!

These methods help define the **interface**. We can test and check the input/output data

### Pythonic way !

In [5]:
class Resistor(object):
    
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0

In [6]:
r1 = Resistor(50e3)
r1.ohms = 10e3

But if we need special behavior of the setter, we can use *@property* decorator

**@property** and **setter**

In [16]:
class VoltageResistance(Resistor):

    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0
        
    @property
    def voltage(self):
        return self.voltage
     
    @voltage.setter
    def voltage(self, voltage):
        if voltage <= 0:
            raise ValueError("%f volts must be > 0" % voltage)
        self._voltage = voltage
        self.current = self._voltage / self.ohms

In [17]:
r2 = VoltageResistance(1e3)
print("Before: %5r amps" %r2.current)
r2.voltage = 10
print("After: %5r amps" %r2.current)

ValueError: 0.000000 volts must be > 0

The setter here also check value that are setted internally !!!!!
```
   self.voltage = 0
```

### Make attributes immutable

**@property** can also be used to make attributes from parent classes immutable

In [56]:
class Resistor(object):
    
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0
        
class FixedResistance(Resistor):
    
    def __init__(self, ohms):
        super().__init__(ohms)
        
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if hasattr(self, '_ohms'):
            raise AttributeError('Can not set Attribute')
        self._ohms = ohms
    

In [57]:
r4 = FixedResistance(10)

ok so far


In [58]:
r4.ohms = 15

AttributeError: Can not set Attribute

This show that we can only set **once** the value of *ohms*

#### Here is another way to declare getter and setter via **property**

In [62]:
class GetterSetterClass(object):
    
    _a = '1 underscore'  # protected
    __b = '2 underscore' # private
    
    def __init__(self, age):
        self.setAge(age)
        
    def getAge(self):
        return self.__age
    
    def setAge(self, age):
        if age < 0:
            self.__age = 0
        else:
            self.__age = age

    age = property(getAge, setAge)

In [63]:
my_object = GetterSetterClass(20)
assert my_object.age == 20

my_object.age = 10
assert my_object.age == 10

my_object.age = -20
assert my_object.age != -20
assert my_object.age == 0

print(my_object._a)
print(my_object.__b)


1 underscore


AttributeError: 'GetterSetterClass' object has no attribute '__b'

## Make sure that we don't set other attributes in getter property methods ! 

In [69]:
class Resistor(object):
    
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0
        
class MysteriousResistor(Resistor):
    
    def __init__(self, ohms):
        super().__init__(ohms)
    
    @property
    def ohms(self):
        self.voltage = self._ohms * self.current
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        self._ohms = ohms
    

In [70]:
r7 = MysteriousResistor(10)
r7.current = 0.01
print("Before: %5r" % r7.voltage)
r7.ohms
print("After: %5r" % r7.voltage)

Before:     0
After:   0.1
